In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint
import json


# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [30]:
wine_location = pd.read_csv("../Output/wine_cleaned.csv")
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [31]:
#Add the latitude  and longitude  empty fields to prime for geocode info
wine_location['latitude'] = ""
wine_location['longitude'] = ""
#wine_location.head()

wine_location_short = wine_location[["country", "province","latitude", "longitude"]]
#wine_grouping = wine_location_short.groupby(["province", "country"])



wine_grouping = pd.DataFrame(wine_location_short)

wine_grouping.drop_duplicates(subset='province', keep='first', inplace=True)

wine_dedup = pd.DataFrame(wine_grouping)

#wine_grouping["country", "province","latitude", "longitude"].unique()
wine_dedup.head()





,country,province,latitude,longitude
0,Italy,Sicily & Sardinia,,
1,Portugal,Douro,,
2,US,Oregon,,
3,US,Michigan,,
5,Spain,Northern Spain,,


In [32]:
# Output File (CSV)
output_data_file = "../Output/duplicate_review.csv"
wine_dedup.to_csv(output_data_file, index=False, header=True)

PermissionError: [Errno 13] Permission denied: '../Output/duplicate_review.csv'

In [33]:
wine_dedup.dtypes

country      object
province     object
latitude     object
longitude    object
dtype: object

In [34]:
#Get the lat and long for the API

# Build the endpoint URL
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'

#Need the province , country from the table and combine together
#Create loop to gather the information

print("Beginning Data Retrieval")
print("-----------------------------")
set=1
# Loop through the list of province & country and perform a request for data on each
for index, row in wine_dedup.iterrows():
    
    #Get the fields from the table
    province_name = row['province']
    country_name = row['country']

    location = (province_name) + ", " + country_name
    #print(combo)
    geo_response = requests.get(geo_url.format(location, g_key)).json()
    #                            & province_name & ", " & country & "&key=" g_key).json()
    #print(geo_response)
    #print(json.dumps(geo_response, indent=4, sort_keys=True))
    try:
        # Extract latitude and longitude
        
        #print(geo_response["results"][0]["geometry"]["location"]["lat"])
        #print(geo_response["results"][0]["geometry"]["location"]["lng"])

        wine_dedup.loc[index, "latitude"] = geo_response["results"][0]["geometry"]["location"]["lat"]
        wine_dedup.loc[index, "longitude"] = geo_response["results"][0]["geometry"]["location"]["lng"]
        
        if index % 100 == 0:
            print(f"Processing Record {index} of location {province_name}, {country_name}.")
        
    except (KeyError):
        print(f"{province_name}, {country_name} not found for Record {index} . Skipping...")
        set+= 1
    except (IndexError):
        print(f"Location not found has Index error {index} for {province_name}, {country_name}. Skipping...")
        set+= 1




Beginning Data Retrieval
-----------------------------
Processing Record 0 of location Sicily & Sardinia, Italy.
Location not found has Index error 16 for Other, Argentina. Skipping...
Location not found has Index error 208 for Coastal Region, South Africa. Skipping...
Processing Record 400 of location Italy Other, Italy.
Location not found has Index error 1186 for Thrace, Turkey. Skipping...
Location not found has Index error 1440 for Wagram-Donauland, Austria. Skipping...
Location not found has Index error 2383 for Terras do Dão, Portugal. Skipping...
Processing Record 2500 of location Panciu, Romania.
Location not found has Index error 2787 for Portuguese Table Wine, Portugal. Skipping...
Location not found has Index error 5082 for Pocerina, Serbia. Skipping...
Location not found has Index error 7586 for Pageon, Greece. Skipping...
Location not found has Index error 7668 for Vinho Espumante de Qualidade, Portugal. Skipping...
Processing Record 9500 of location Sebes, Romania.
Locati

In [35]:
wine_dedup.head()

,country,province,latitude,longitude
0,Italy,Sicily & Sardinia,37.6,14.0154
1,Portugal,Douro,41.5013,-5.51229
2,US,Oregon,43.8041,-120.554
3,US,Michigan,44.3148,-85.6024
5,Spain,Northern Spain,43.3366,-3.20272


In [36]:
# Combine the data into a single dataset

wine_location.drop(['latitude', 'longitude'], axis=1, inplace=True) 
                 
#wine_location.head()
wine_location_complete = pd.merge(wine_location, wine_dedup,  how='left', 
                                  left_on=['province','country'], right_on = ['province','country'])

wine_location_complete.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,37.6,14.0154
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,41.5013,-5.51229
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,43.8041,-120.554
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,44.3148,-85.6024
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,43.8041,-120.554


In [39]:
# Output File (CSV)
output_data_file = "../Output/wine_cleaned_location.csv"
wine_location_complete.to_csv(output_data_file, index=False, header=True)

In [41]:

#wine_location = wine_location.replace('', np.nan)
#wine_location = wine_location.dropna()
#wine_location.head()

###Remove data when the lat and long data is blank
wine_location_complete = wine_location_complete.dropna(subset=['latitude', 'longitude'])

##Average Points per province & country

##Color change for sizing of the points

In [43]:
# Store latitude and longitude in locations
locations = wine_location_complete[["latitude", "longitude"]]

points = wine_location_complete["points"]

from ipywidgets.embed import embed_minimal_html

In [44]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")
#print(fig)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=points, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
#print(heat_layer)

# Customize the size of the figure


figure_layout = {
    'width': '1500px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '12px',
    'margin': '0 auto 0 auto'
}

#1500 x 2100
fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


TraitError:  is not a valid latitude. Latitudes must be floats

In [45]:
# Save the figure in html
embed_minimal_html('Output/WinePoints3.html', views=[fig])

FileNotFoundError: [Errno 2] No such file or directory: 'Output/WinePoints3.html'